<a href="https://colab.research.google.com/github/Nitish-37/Route-optimization-for-last-mile-delivery/blob/main/Optimation_Route_for_last_mile_delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [8]:
def load_distance_matrix(file_path):
    """Reads distance matrix from Excel file"""
    df = pd.read_excel(file_path, index_col=0)
    return df.values.tolist()

In [9]:
distance_matrix = load_distance_matrix('df_distance_matrix.xlsx')

In [10]:
def create_data_model(distance_matrix):
    """Structures the problem data"""
    return {
        'distance_matrix': distance_matrix,
        'num_vehicles': 1,  # Single delivery vehicle
        'depot': 0           # Starting/ending point (Location 0)
    }

In [11]:
def initialize_routing(data):
    """Sets up routing index manager and model"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )

    routing = pywrapcp.RoutingModel(manager)
    return manager, routing

In [12]:
def add_distance_constraint(routing, manager, data):
    """Adds distance callback function"""
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [13]:
def set_search_parameters():
    """Sets optimization parameters"""
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(5)
    return search_parameters

In [14]:
def print_solution(manager, routing, solution):
    """Prints optimized route details"""
    index = routing.Start(0)
    plan_output = 'Optimal Delivery Route:\n'
    route_distance = 0
    route = []

    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        plan_output += f'Location {node} -> '
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)

    plan_output += f'Location {manager.IndexToNode(index)}\n'
    print(plan_output)
    print(f'Total Route Distance: {route_distance} units')
    print(f'Route Sequence: {route + [0]}')

In [17]:
def main():
    # Load data
    distance_matrix = load_distance_matrix('df_distance_matrix.xlsx')
    data = create_data_model(distance_matrix)

    # Initialize routing
    manager, routing = initialize_routing(data)

    # Add constraints
    add_distance_constraint(routing, manager, data)

    # Configure solver
    search_parameters = set_search_parameters()

    # Solve problem
    solution = routing.SolveWithParameters(search_parameters)

    # Print results
    if solution:
        print_solution(manager, routing, solution)
    else:
        print("No solution found!")
if __name__ == '__main__': # Changed _name_ to __name__
    main()


Optimal Delivery Route:
Location 0 -> Location 9 -> Location 5 -> Location 8 -> Location 6 -> Location 2 -> Location 10 -> Location 16 -> Location 14 -> Location 13 -> Location 12 -> Location 11 -> Location 15 -> Location 3 -> Location 4 -> Location 1 -> Location 7 -> Location 0

Total Route Distance: 4384 units
Route Sequence: [0, 9, 5, 8, 6, 2, 10, 16, 14, 13, 12, 11, 15, 3, 4, 1, 7, 0]
